## Prophet Model

## Imports

In [ ]:
# Import the required libraries and dependencies
import warnings
import pandas as pd
import numpy as np
import hvplot.pandas
from prophet import Prophet
from pathlib import Path
from plotting import Plotter
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
# Ignoring warnings
warnings.filterwarnings('ignore')

## Data

In [ ]:
# Setting global variables
tickers = ['ARKK','SPY','FNGU']

In [ ]:
# Reading in ETF data to DataFrame
etf_data = pd.read_csv(Path("Resources/Data/etf_data.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True)

In [ ]:
# Separating ETF DataFrame into separate DataFrames
arkk,spy,fngu = [etf_data[i].to_frame(i) for i in tickers]

## Trian Test Split

In [ ]:
# Method for train test split
def train_test_split(df):
    # Creating copy of DataFame and resetting the index
    data = df.copy().reset_index()
    # Changing column names
    data.columns = ['ds','y']
    # Returning train and test daa
    return data[:-60],data[-60:]

In [ ]:
# Splitting data into train and test
arkk_train,arkk_test = train_test_split(arkk)
spy_train,spy_test = train_test_split(spy)
fngu_train,fngu_test = train_test_split(fngu)

## Create Model, Train Model & Forecast

In [ ]:
# Method to create, train and forecast models
def model_forecast(train,test):
    # Create Prophet model
    model = Prophet()
    
    # Train Prophet model
    model.fit(train)
    
    # Saving forecacsts
    forecast = model.predict(test)[['ds','yhat_lower']]

    # Returning forecast
    return forecast

### Forecasting

In [ ]:
# Saving forecast
arkk_forecast, spy_forecast, fngu_forecast = [model_forecast(i[0],i[1]) for i in [(arkk_train,arkk_test),(spy_train,spy_test),(fngu_train,fngu_test)]]

## Evaluation

In [ ]:
def evaluate(ticker, forecast, test):
    # Converting testing data and forecast to numpy arrays
    test,forecast = [np.array(i) for i in [test['y'],forecast['yhat_lower']]]
    
    # Saving each metric as variable
    mae = round(mean_absolute_error(test,forecast),2)
    mse = round(mean_squared_error(test,forecast),2)
    rmse = round(mse**.5,2)
    
    # Printing ETF ticker
    print(ticker)
    # Printing metrics
    print(f'Mean Absolute Error: {mae}')
    print(f'Mean Squared Error: {mse}')
    print(f'Root Mean Squared Error: {rmse}')
    
    # Returning metrics
    return [mae,mse,rmse]

### ARKK

In [ ]:
# Saving and printing ARKK metrics
arkk_mae, arkk_mse, arkk_rmse = evaluate(tickers[0],arkk_forecast,arkk_test)

### SPY

In [ ]:
# Saving and printing SPY metrics
spy_mae, spy_mse, spy_rmse = evaluate(tickers[1],spy_forecast,spy_test)

### FNGU

In [ ]:
# Saving and printing FNGU metrics
fngu_mae, fngu_mse, fngu_rmse = evaluate(tickers[2],fngu_forecast,fngu_test)

In [ ]:
# Instantiating custom Plotter class
plotter = Plotter('Prophet')

In [ ]:
# Lambda helper method to create DataFrame of error metrics
get_error_df = lambda a,s,f: pd.DataFrame({tickers[0]:a,tickers[1]:s,tickers[2]:f}, index=[0])

### Mean Absolute Error

In [ ]:
# Plotting MAS bar plot
plotter.bar(get_error_df(arkk_mae,spy_mae,fngu_mae),'Mean Absolute Error')

### Mean Squared Error

In [ ]:
# Plotting MSE bar plot
plotter.bar(get_error_df(arkk_mse,spy_mse,fngu_mse),'Mean Squared Error')

### Root Mean Squared Error

In [ ]:
# Plotting RMSE bar plot
plotter.bar(get_error_df(arkk_rmse,spy_rmse,fngu_rmse),'Root Mean Squared Error')

## Results

### Actual vs Forecast

In [ ]:
# Helper method to display DataFrame head and tail
display_head_tail = lambda df: display(df.head(),df.tail())
# Variable for titles
avf = 'Actual vs Forecasted'
# Helper method to get titles
get_avf_title = lambda index: tickers[index] + ' ' + avf

In [ ]:
# Helper method to get actual and forecasted adjusted closing prices as DataFrame
def get_actual_forecast(ticker,X_test,forecast):
    # Setting index as 'ds'
    test,forecasted = [i.copy().set_index('ds') for i in [X_test,forecast]]
    
    # Joining DataFrames
    df = pd.concat([test,forecasted], join='inner', axis=1)
    # Rounding yhat_lower column to two decimal places
    df['yhat_lower'] = df['yhat_lower'].apply(lambda x: round(x,2))
    # Renaming columns
    df.rename(columns={'yhat_lower': f'{ticker} Forecasted', 'y': f'{ticker} Actual'}, inplace=True)
    # Renaming index
    df.index.rename('Date',inplace=True)
    
    # Returning DataFrame
    return df

In [ ]:
# Saving and displaying ARKK actual and forecasted adjusted closing prices
arkk_actual_forecasted = get_actual_forecast(tickers[0],arkk_test,arkk_forecast)
display_head_tail(arkk_actual_forecasted)

In [ ]:
# Plotting ARKK actual vs forecasted adjusted closing prices
plotter.line(arkk_actual_forecasted, get_avf_title(0)) 

In [ ]:
# Saving and displaying SPY actual and forecasted adjusted closing prices
spy_actual_forecasted = get_actual_forecast(tickers[1],spy_test,spy_forecast)
display_head_tail(spy_actual_forecasted)

In [ ]:
# Plotting SPY actual vs forecasted adjusted closing prices
plotter.line(spy_actual_forecasted, get_avf_title(1)) 

In [ ]:
# Saving and displaying FNGU actual and forecasted adjusted closing prices
fngu_actual_forecasted = get_actual_forecast(tickers[2],fngu_test,fngu_forecast)
display_head_tail(fngu_actual_forecasted)

In [ ]:
# Plotting FNGU actual vs forecasted adjusted closing prices
plotter.line(fngu_actual_forecasted, get_avf_title(2)) 

In [ ]:
# Saving and displaying ETF actual and forecasted adjusted closing prices
actual_forecasted = pd.concat([arkk_actual_forecasted,spy_actual_forecasted,fngu_actual_forecasted], axis=1, join='inner')
display_head_tail(actual_forecasted)

In [ ]:
# Plotting ETF actual vs forecasted adjusted closing prices
plotter.line(actual_forecasted, avf, None)

### Training Data vs Actual vs Forecast

In [ ]:
# Creating DataFrame of training, actual and forecasted adjusted cloding prices
training_data = etf_data[:-60].rename(columns={tickers[i]: tickers[i] + ' Training Data' for i in range(3)})
train_actual_forecasted = pd.concat([training_data,actual_forecasted])

In [ ]:
# Helper method to get DataFrame by ticker
get_df_by_ticker = lambda ticker: train_actual_forecasted[[col for col in train_actual_forecasted if col.startswith(ticker)]]

In [ ]:
# Variable for title and lambda helper method to get title
taf = 'Training Data vs Actual vs Forecasted'
get_train_title = lambda index: tickers[index] + ' ' + taf

In [ ]:
# Saving and displaying ARKK data
arrk_train_actual_forecasted = get_df_by_ticker(tickers[0])
display_head_tail(arrk_train_actual_forecasted)

In [ ]:
# Plotting ARKK data
plotter.line(arrk_train_actual_forecasted, get_train_title(0))

In [ ]:
# Saving and displaying SPY data
spy_train_actual_forecasted = get_df_by_ticker(tickers[1])
display_head_tail(spy_train_actual_forecasted)

In [ ]:
# Plotting SPY data
plotter.line(spy_train_actual_forecasted, get_train_title(1))

In [ ]:
# Saving and displaying FNGU data
fngu_train_actual_forecasted = get_df_by_ticker(tickers[2])
display_head_tail(fngu_train_actual_forecasted)

In [ ]:
# Plotting FNGU data
plotter.line(fngu_train_actual_forecasted, get_train_title(2))

In [ ]:
# Displaying ETF data
display_head_tail(train_actual_forecasted)

In [ ]:
# Plotting ETF data
plotter.line(train_actual_forecasted, taf, None)